In [ ]:
!pip install -qq -U mlxtend

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from tqdm.auto import tqdm

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
STOPWORDS = set(nltk.corpus.stopwords.words("english") + ["says", "us", "said" ,"new"])
PORTER_STEMMER = PorterStemmer()

In [ ]:
reuters_4000 = pd.read_csv("/content/drive/MyDrive/Atharva_Scrape/Reuters-100000.csv")

In [ ]:
def clean_tweets(x):
    # Lowercase
    sentence = x.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub('http[s]?://\S+', '', sentence)
    sentence = re.sub(r'([^a-zA-Z ]+?)', '', sentence)
    #print(sentence)
    #sentence = sentence.replace('\n', '')
    # Remove URLs
    
    #print(sentence)
    # Remove double spacing
    #sentence = re.sub('\s+', ' ', sentence)
    tokenized_tweet = [word for word in word_tokenize(sentence) if word not in STOPWORDS]
    tokenized_tweet = [PORTER_STEMMER.stem(word) for word in tokenized_tweet]
    return tokenized_tweet

In [ ]:
tqdm.pandas()
reuters_4000["tokenized_tweet"] = reuters_4000["tweet"].progress_apply(lambda x : clean_tweets(x))

  0%|          | 0/39912 [00:00<?, ?it/s]

In [ ]:
tokenized_tweets = reuters_4000["tokenized_tweet"].values.tolist()
te = TransactionEncoder()
te_ary = te.fit(tokenized_tweets).transform(tokenized_tweets)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
frequent_itemsets = apriori(df, min_support=0.008, use_colnames=True)

In [ ]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [ ]:
frequent_itemsets[frequent_itemsets['length'] >= 2]

support                 itemsets  length
239  0.013029            (beij, olymp)       2
240  0.008920           (winter, beij)       2
241  0.009972          (biden, presid)       2
242  0.013404            (covid, case)       2
243  0.011876          (vaccin, covid)       2
244  0.009822             (hong, kong)       2
245  0.012001          (russia, invas)       2
246  0.008970         (russian, invas)       2
247  0.018015          (ukrain, invas)       2
248  0.008920          (winter, olymp)       2
249  0.009997         (ukrain, presid)       2
250  0.009621          (putin, ukrain)       2
251  0.013805       (russia, sanction)       2
252  0.040690         (russia, ukrain)       2
253  0.022976        (russian, ukrain)       2
254  0.008844            (state, unit)       2
255  0.010648            (war, ukrain)       2
256  0.010799  (russia, ukrain, invas)       3